# Intro to Deep Learning - PyTorch Lightning

https://towardsdatascience.com/converting-from-keras-to-pytorch-lightning-be40326d7b7d

## Install packages

In [1]:
!pip install torch torchvision torchmetrics pytorch-lightning

In [2]:
import os

import pandas as pd
import seaborn as sn
import torch
from IPython import display
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import torchmetrics
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 128

## Prepare train-test splits

In [3]:
# Init DataLoader from MNIST Dataset
train_ds = MNIST(PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

test_ds = MNIST(PATH_DATASETS, train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

## Define the model

In [4]:
class MNISTModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.accuracy = torchmetrics.Accuracy()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        self.log('train_loss_step', loss)
        self.accuracy(preds, y)
        self.log('train_acc_step', self.accuracy)
        return loss

    def test_step(self, batch, batch_nb):
        x, y = batch
        preds = self(x)
        loss = F.cross_entropy(preds, y)
        self.log('test_loss_step', loss)
        self.accuracy(preds, y)
        self.log('test_acc_step', self.accuracy)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

## Compile and train the model

In [5]:
# Init our model
mnist_model = MNISTModel()

# Initialize a trainer
trainer = Trainer(
    max_epochs=3,
    limit_train_batches=100,
    callbacks=[TQDMProgressBar(refresh_rate=10)]
)

# Train the model ⚡
trainer.fit(mnist_model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 7.9 K 
--------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)
/Users/rbates/src/adc2022-foss-ds-ml-tools/ENV/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

2022-09-15 19:24:20.581810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
`Trainer.fit` stopped: `max_epochs=3` reached.


## Evaluate the model

In [6]:
trainer.test(mnist_model, test_loader)

/Users/rbates/src/adc2022-foss-ds-ml-tools/ENV/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_acc_step         0.6431833505630493
     test_loss_step         1.1750569343566895
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_step': 1.1750569343566895, 'test_acc_step': 0.6431833505630493}]